In [1]:
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format

import sys
sys.path.append('../lib/')
from financelib import FinLoad
from financelib import FinCalc
from financelib import FinFetch
from financelib import FinPlot
from financelib import FinInvestmentsGet

from pathlib import Path

import requests # for investments yahoo finance data

import time # sleep fetch

In [2]:
YEAR = 2024
data_path_o = Path("../../tmp/data")

### Load Data 2024

In [3]:
init_holdings = FinLoad.load_init_holdings(data_path_o, YEAR)
df_year_cashflow = FinLoad.load_cashflow(data_path_o, YEAR)
df_year_investments = FinLoad.load_investments(data_path_o, YEAR)

### Calculate

In [4]:
df_m_cashflow = FinCalc.calc_monthly_cashflow(df_year_cashflow)

In [5]:
df_m_cashflow.set_index("Date").T

Date,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31
incomes,"2,031.00","1,642.00","1,630.00","1,642.00","1,682.00","1,688.00","2,534.34","1,694.00","1,694.00","1,778.00","1,693.00","2,757.00"
liabilities,"-1,069.21",-878.36,-359.75,-936.96,-614.41,-842.75,-386.74,-389.54,-621.00,-569.47,-544.63,-478.20
savings,961.79,763.64,"1,270.25",705.04,"1,067.59",845.25,"2,147.60","1,304.46","1,073.00","1,208.53","1,148.37","2,278.80"
saving_rate,0.47,0.47,0.78,0.43,0.63,0.50,0.85,0.71,0.63,0.68,0.68,0.83


In [6]:
df_m_cashflow

,Date,incomes,liabilities,savings,saving_rate
0,2024-01-31,"2,031.00","-1,069.21",961.79,0.47
1,2024-02-29,"1,642.00",-878.36,763.64,0.47
2,2024-03-31,"1,630.00",-359.75,"1,270.25",0.78
3,2024-04-30,"1,642.00",-936.96,705.04,0.43
4,2024-05-31,"1,682.00",-614.41,"1,067.59",0.63
5,2024-06-30,"1,688.00",-842.75,845.25,0.50
6,2024-07-31,"2,534.34",-386.74,"2,147.60",0.85
7,2024-08-31,"1,694.00",-389.54,"1,304.46",0.71
8,2024-09-30,"1,694.00",-621.00,"1,073.00",0.63
9,2024-10-31,"1,778.00",-569.47,"1,208.53",0.68


In [7]:
fig_cashflow = FinPlot.plot_cashflow(df_m_cashflow)
fig_cashflow.show()

In [8]:
# Liquidity in Hype, Revolut
current_balance_revolut = round(df_year_cashflow.loc[df_year_cashflow["Type"] == "Revolut"]['Qty'].sum(),2) + init_holdings['liquidity']['Revolut - EUR']
current_balance_hype    = round(df_year_cashflow.loc[df_year_cashflow["Type"] == "Hype"]['Qty'].sum(),2) + init_holdings['liquidity']['Hype - EUR']
current_balance_cash    = round(df_year_cashflow.loc[df_year_cashflow["Type"] == "Cash"]['Qty'].sum(),2) + init_holdings['liquidity']['Money - EUR']
current_balance_bitget  = round(df_year_cashflow.loc[df_year_cashflow["Type"] == "Bitget"]['Qty'].sum(),2)

In [9]:
print("Current LIQUIDITY")
print(f"Revolut: {current_balance_revolut}")
print(f"Hype: {current_balance_hype}")
print(f"Cash: {current_balance_cash}")
print(f"Bitget Fiat: {current_balance_bitget}")

Current LIQUIDITY
Revolut: 28.61
Hype: 14382.58
Cash: 300.0
Bitget Fiat: 0.0


### Detail Expenses January 2025

In [10]:
df_expenses = FinCalc.calc_expenses(df_year_cashflow[ df_year_cashflow.index.month == 6 ])
df_expenses["Qty"] = df_expenses.Qty.abs() # sunburst does not understand negative values
fig = FinPlot.plot_expenses_donut(df_expenses)
fig.show()

### Investments 2024

In [11]:
init_holdings = FinLoad.load_init_holdings(data_path_o, YEAR)
df_year_investments = FinLoad.load_investments(data_path_o, YEAR)

In [12]:
df_init_investments = FinInvestmentsGet.get_init_holdings_to_df(init_holdings, YEAR)
df_year_investments = pd.concat([df_init_investments, df_year_investments])


In [13]:
df_year_investments

,Type,Symbol,Qty,Category,Subcategory,Description
Date,,,,,,
2024-01-01,Cryptocurrencies,SOL,4.86,Init,Holdings,From previous year
2024-01-01,Cryptocurrencies,ETH,0.01,Init,Holdings,From previous year
2024-01-01,ETFs,IE00BK5BQT80,9.00,Init,Holdings,From previous year
2024-01-01,ETFs,JE00B1VS3770,1.00,Init,Holdings,From previous year
2024-08-13,Cryptocurrencies,USDT,108.24,Buy,Holdings,Bought 108 USDT for Futures account


In [14]:
holdings_monthlyized = FinInvestmentsGet.get_holdings_monthlyized(df_year_investments, YEAR)

In [15]:
holdings_monthlyized['Cryptocurrencies']['ETH']

,Qty,CumQty
2024-01-31,0.01,0.01
2024-02-29,0.00,0.01
2024-03-31,0.00,0.01
2024-04-30,0.00,0.01
2024-05-31,0.00,0.01
2024-06-30,0.00,0.01
2024-07-31,0.00,0.01
2024-08-31,0.00,0.01
2024-09-30,0.00,0.01
2024-10-31,0.00,0.01


In [16]:
holdings_monthlyized.keys()

dict_keys(['Cryptocurrencies', 'ETFs'])

In [17]:
assets_monthlyized = FinInvestmentsGet.get_assets_monthlyized(holdings_monthlyized, data_path_o, YEAR)

SOL
../../tmp/data/2024/investments/exchange/SOL-EUR.csv already exists. Data Loaded from local.
ETH
../../tmp/data/2024/investments/exchange/ETH-EUR.csv already exists. Data Loaded from local.
USDT
../../tmp/data/2024/investments/exchange/USDT-EUR.csv already exists. Data Loaded from local.
IE00BK5BQT80
../../tmp/data/2024/investments/exchange/IE00BK5BQT80-EUR.csv already exists. Data Loaded from local.
JE00B1VS3770
../../tmp/data/2024/investments/exchange/JE00B1VS3770-EUR.csv already exists. Data Loaded from local.


In [18]:
assets = FinInvestmentsGet.get_assets_global(assets_monthlyized, holdings_monthlyized)

In [19]:
assets['ETFs'].keys()

dict_keys(['IE00BK5BQT80', 'JE00B1VS3770'])

In [24]:
assets['Cryptocurrencies']['USDT'].T

,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31
Open,0.90,0.92,0.92,0.93,0.93,0.92,0.93,0.92,0.90,0.90,0.92,0.95
Close,0.92,0.92,0.93,0.93,0.92,0.93,0.92,0.90,0.90,0.92,0.95,0.96
Returns,NaN,0.00,0.01,0.00,-0.01,0.01,-0.01,-0.02,0.00,0.02,0.03,0.01
Qty,0.00,0.00,0.00,0.00,0.00,0.00,0.00,108.24,0.00,0.00,0.00,0.00
CumQty,0.00,0.00,0.00,0.00,0.00,0.00,0.00,108.24,108.24,108.24,108.24,108.24
Holdings,0.00,0.00,0.00,0.00,0.00,0.00,0.00,97.42,97.42,99.58,102.83,103.91


In [34]:
cryptocurrency_year_holdings = 0
for symbol in symbols:
    df_symbol_holdings = getSymbolHoldings("Cryptocurrencies", symbol, list_investments_asset_class, init_holdings, year)
    cryptocurrency_year_holdings += df_symbol_holdings["holdings €"]

In [35]:
cryptocurrency_year_holdings

Date
2024-01-30     462.44
2024-02-28     601.79
2024-03-31     952.95
2024-04-30     611.48
2024-05-31     783.76
2024-06-30     702.00
2024-07-31     807.59
2024-08-31     721.44
2024-09-30     791.55
2024-10-31     880.07
2024-11-30   1,236.90
2024-12-31   1,030.07
Name: holdings €, dtype: float64

In [36]:
# Then SUM on NW

### NW 2024

In [37]:
init_holdings['liquidity']

{'Hype - EUR': 6, 'Revolut - EUR': 16.74, 'Money - EUR': 14.13}

In [38]:
liquidity = df_m_cashflow["savings"].values.cumsum() + init_holdings['liquidity']['Hype - EUR'] + init_holdings['liquidity']['Revolut - EUR'] + init_holdings['liquidity']['Money - EUR']

In [39]:
list(cryptocurrency_year_holdings.values)

[np.float64(462.44376800000003),
 np.float64(601.7949739999999),
 np.float64(952.9486120000001),
 np.float64(611.479244),
 np.float64(783.7628880000001),
 np.float64(702.000634),
 np.float64(807.592586),
 np.float64(721.4430884000001),
 np.float64(791.5517060000001),
 np.float64(880.0685808000001),
 np.float64(1236.8958344),
 np.float64(1030.0694224)]

In [40]:
liquidity

array([  998.66 ,  1762.3  ,  3032.55 ,  3737.59 ,  4805.18 ,  5650.43 ,
        7798.03 ,  9102.49 , 10175.491, 11384.021, 12532.391, 14811.191])

In [41]:
liquidity_end_2023 = 24.30
crypto_end_2023 = 473.56
columns = ["liquidity", "crypto"]

nw2023 = pd.DataFrame(data = zip([liquidity_end_2023],[crypto_end_2023]), columns = columns, index = pd.date_range(f'2023-12-31',f'2023-12-31', freq='ME'))

In [42]:
zipped = zip(liquidity, cryptocurrency_year_holdings.values)
columns = ["liquidity", "crypto"]
nw2024 = pd.DataFrame(index = pd.date_range(f'{year}-01-01',f'{year}-12-31', freq='ME'), data = zipped, columns = columns)

In [43]:
nw = pd.concat([nw2023, nw2024])

In [44]:
nw["nw"] = nw.liquidity + nw.crypto

In [45]:
nw["nwch"] = (nw["nw"] - nw.nw.shift(1) )
nw["nw%"] = (nw["nw"] - nw.nw.shift(1) )/ nw["nw"]

In [46]:
nw.T

,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31
liquidity,24.30,998.66,"1,762.30","3,032.55","3,737.59","4,805.18","5,650.43","7,798.03","9,102.49","10,175.49","11,384.02","12,532.39","14,811.19"
crypto,473.56,462.44,601.79,952.95,611.48,783.76,702.00,807.59,721.44,791.55,880.07,"1,236.90","1,030.07"
nw,497.86,"1,461.10","2,364.09","3,985.50","4,349.07","5,588.94","6,352.43","8,605.62","9,823.93","10,967.04","12,264.09","13,769.29","15,841.26"
nwch,NaN,963.24,902.99,"1,621.40",363.57,"1,239.87",763.49,"2,253.19","1,218.31","1,143.11","1,297.05","1,505.20","2,071.97"
nw%,NaN,0.66,0.38,0.41,0.08,0.22,0.12,0.26,0.12,0.10,0.11,0.11,0.13


### FIRE Metrics on end 2024

In [47]:
current_yearly_expediture = df_m_cashflow.liabilities.sum()
current_nw = nw.nw.iloc[-1]

In [48]:
desired_wd = 0.030
fu_number = abs(current_yearly_expediture) / desired_wd
print(fu_number)

256367.3


In [49]:
progress_fi = current_nw / fu_number
print(progress_fi)

0.06179126753841071


In [50]:
current_yearly_saved = nw.nwch.sum()

In [51]:
current_yearly_monthly_savings = current_yearly_saved/12

In [52]:
months_left_to_fire = (fu_number - current_nw)/current_yearly_monthly_savings
print(f"Months left to FIRE: {int(months_left_to_fire)}")

Months left to FIRE: 188
